In [1]:
import pandas as pd


In [2]:
import requests
from bs4 import BeautifulSoup
import bs4
import time
import numpy as np
import re
import time
#the goal is scrapping wikipedia in order to obtain information about citrus fruit
#We take the description section of each one and then we will extract main info.


In [8]:
page_citrus = 'https://en.wikipedia.org/wiki/List_of_citrus_fruits'
page_salads = 'https://en.wikipedia.org/wiki/List_of_leaf_vegetables'
page_spices = 'https://en.wikipedia.org/wiki/List_of_culinary_herbs_and_spices'
page_fruit = 'https://simple.wikipedia.org/wiki/List_of_fruits'
page_herbs = 'https://simple.wikipedia.org/wiki/List_of_herbs'
page_vegetable = 'https://simple.wikipedia.org/wiki/List_of_vegetables'



In [48]:

#A function that gets the URL of the page to be scraped
#,gets the html content and uses BeautifulSoup to parse html content

def make_soup(link):
    get_page = requests.get(link)
    html = get_page.content
    soup = BeautifulSoup(html, 'html.parser')
    return  soup


#####This function create a list with all the link of the foods in a wikipedia Page.
def make_link_list(wiki_page_to_scrap):
    start_time = time.time()
    link_table = []
    soup = make_soup(wiki_page_to_scrap)
    table = soup.find('table',{'class':'wikitable'})
    
            ### This first loop is used to scrap Wiki table Data.
    if isinstance(table , bs4.element.Tag):        
    
        table_cells = []
        table = soup.find('table',{'class':'wikitable'})
        for row in table.find_all("tr"):
            cells = row.find_all(['th' , 'td'])
            table_cells.append(cells)
        
        ### This loop is used to locate the "Common name" column index in our table cells
        indices = []
        for j in table_cells:
            for i, elem in enumerate(j):
                elem = str(elem)
                if 'name' in elem:
                    indices.append(i)
        indice = indices[0]
        
        #print('Table Cells' , table_cells[:3])
        ### Here we implement a loop to keep only the string of the Common name column.
        
        for cell in table_cells[2:]:
            
            if (len(cell) < indice) == True :
                pass
             
            else:    
                link_table.append(cell[indice].text)
            
        ### Let's clean the table.
        for link in range(len(link_table)):
            link_table[link] = link_table[link].strip('\n')
            
        print('cpu time for the table schema = {:.4f} sec.'.format(time.time() - start_time))
            
            ### Here is when the Wiki page is just an Alphabetical List.    
    elif (len(soup.find_all('div' , {'class':'div-col'}))>0) == True : 
        print(len(soup.find_all('div' , {'class':'div-col'})))
        for row in soup.find_all('div' , {'class':'div-col'}):
            
            for col in row.find_all('li'):
                species = col.text
                ###We just keep the common name of the species, because only the common name is used in recipes.
                only_common_species = re.sub("[\(\[].*?[\)\]]", "", species) 
                only_common_species , sep , tail = only_common_species.partition(',')
                link_table.append(only_common_species)
                
                ###Cleaning of the list, we remove all the occurence of string begining by List.
        for word in link_table[:]:
            if word.find('List') != -1:
                link_table.remove(word)
            
            
        print('cpu time for the  Alphabetical list schema = {:.4f} sec.'.format(time.time() - start_time))  
        
        ### For the list pattern without alphabetical list.
    elif (len(soup.find_all('div' , {'class' : 'mw-parser-output'}))> 0 ) == True:
        for row in soup.find_all('div' , {'class' : 'mw-parser-output'}):
             for col in row.find_all('li'):
                    species = col.text
        ###We just keep the common name of the species, because only the common name is used in recipes.
                    only_common_species = re.sub("[\(\[].*?[\)\]]", "", species) 
                    only_common_species , sep , tail = only_common_species.partition(',')
                    link_table.append(only_common_species)
                    
                ###Cleaning of the list, we remove all the occurence of string begining by List.
        for word in link_table[:]:
            if word.find('List') != -1:
                link_table.remove(word)
        
        print('cpu time for the list schema = {:.4f} sec.'.format(time.time() - start_time))            

    return link_table  



list_fruit = make_link_list(page_fruit)
number_fruit = len(list_fruit)
print('Number of fruits', len(list_fruit))
print(list_fruit)

2


AttributeError: 'list' object has no attribute 'partition'

In [9]:

list_citrus = make_link_list(page_citrus)
number_citrus = len(list_citrus)
print('Number of citrus' , len(list_citrus))


list_salad = make_link_list(page_salads)
number_salad = len(list_salad)
print('Number of salads' , len(list_salad))


list_spices = make_link_list(page_spices)
number_spices = len(list_spices)
print('Number of spices' , len(list_spices))


list_fruit = make_link_list(page_fruit)
number_fruit = len(list_fruit)
print('Number of fruits', len(list_fruit))


list_herbs = make_link_list(page_herbs)
number_herbs = len(list_herbs)
print('Number of herbs', len(list_herbs))


list_vegetable = make_link_list(page_vegetable)
number_vegetable = len(list_vegetable)
print('Number of vegetables' , number_vegetable)



print('Total number' , number_citrus + number_fruit + number_herbs + number_salad + number_spices + number_vegetable )

cpu time for the table schema = 0.5810 sec.
Number of citrus 50
cpu time for the table schema = 2.1336 sec.
Number of salads 438
20
cpu time for the  Alphabetical list schema = 0.7485 sec.
Number of spices 203
2
cpu time for the  Alphabetical list schema = 3.5630 sec.
Number of fruits 114
1
cpu time for the  Alphabetical list schema = 0.4125 sec.
Number of herbs 46
cpu time for the list schema = 0.4438 sec.
Number of vegetables 132
Total number 983


In [24]:
# Let's create a dictionary with the different categories of vegetables. 

### This function is the constructor of the dictionnary, when we want to add a category and a list (which can be empty)to the dict.
def add_cat_to_dict(list_of_ingr , Category_name , food_dict , existency = True):
    #existency argument is the signal to ceate a new dictionnary.
    if existency == True:
        CAT_NAME = Category_name.upper()    
        food_dict.update({CAT_NAME : list_of_ingr})
    else :
        food_dict = dict()
        CAT_NAME = Category_name.upper()    
        food_dict.update({CAT_NAME : list_of_ingr})        
    return

###This function permits to the user to add an element in a category, I thought that the user will add ingredient
###one by one, so he should put one tuple (category, ingredient) as input:
#def add_instance
    

ingre_dict = dict()
print(len(ingre_dict))

add_to_dict(list_fruit , 'Fruits',ingre_dict)
print(ingre_dict)


0
{'FRUITS': ['Açaí', 'Apple', 'Akee', 'Apricot', 'Avocado', 'Banana', 'Bilberry', 'Blackberry', 'Blackcurrant', 'Black sapote', 'Blueberry', 'Boysenberry', "Buddha's hand ", 'Crab apples', 'Currant', 'Cherry', 'Cherimoya ', 'Chico fruit', 'Cloudberry', 'Coconut', 'Cranberry', 'Cucumber', 'Damson', 'Date', 'Dragonfruit ', 'Durian', 'Elderberry', 'Feijoa', 'Fig', 'Goji berry', 'Gooseberry', 'Grape\nRaisin', 'Raisin', 'Grapefruit', 'Guava', 'Honeyberry', 'Huckleberry', 'Jabuticaba', 'Jackfruit', 'Jambul', 'Japanese plum', 'Jostaberry', 'Jujube', 'Juniper berry', 'Kiwano ', 'Kiwifruit', 'Kumquat', 'Lemon', 'Lime', 'Loquat', 'Longan', 'Lychee', 'Mango', 'Mangosteen', 'Marionberry', 'Melon\nCantaloupe\nHoneydew\nWatermelon', 'Cantaloupe', 'Honeydew', 'Watermelon', 'Miracle fruit', 'Mulberry', 'Nectarine', 'Nance', 'Olive', 'Orange\nBlood orange\nClementine\nMandarine\nTangerine', 'Blood orange', 'Clementine', 'Mandarine', 'Tangerine', 'Papaya', 'Passionfruit', 'Peach', 'Pear', 'Persimmon', 

In [21]:
page_vegetable = 'https://simple.wikipedia.org/wiki/List_of_vegetables'
list_vegetable = make_link_list(page_vegetable)
print('Number of vegetables' , len(list_vegetable))

cpu time for the list schema = 0.1300 sec.
Number of vegetables 0


In [13]:
page_fruit = 'https://simple.wikipedia.org/wiki/List_of_fruits'

list_fruit = make_link_list(page_fruit)
print('List of fruit' , list_fruit[:4] , len(list_fruit))

cpu time for the list schema = 0.1417 sec.
List of fruit ['Açaí', 'Apple', 'Akee', 'Apricot'] 114


In [42]:
a='PastiS\nRicard'
a= a.split('\n')
print(a)

['PastiS', 'Ricard']


In [46]:
list_fruit = make_link_list(page_fruit)
number_fruit = len(list_fruit)
print('Number of fruits', len(list_fruit))
print(list_fruit)

2


AttributeError: 'list' object has no attribute 'find'